## Flow of Information and Cascading Behavior Through the Western States Power Grid

### Tucker Knaak - Department of Mathematics, Creighton University - 2023

#### The Louvain Community Detection Algorithm is heuristic method used to extract the community structure of a network.  The global and local efficiencies as well as the edge and node betweenness centralities of a network determine how information flows through the network.  The Western States Power Grid of the United States encompasses eleven US states and two Canadian provinces west of the Rocky Mountains.  In this code, we will investigate how the deletion of edges (power lines) and nodes (power stations) in Louvain communities of the network affects the flow of information through the network as well as how susceptible the network is to a cascade of failures.

In [1]:
import matplotlib.transforms as mtransforms
import matplotlib.animation as animation
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import community
from IPython.display import HTML
from tabulate import tabulate
np.warnings.filterwarnings('ignore', category=np.VisibleDeprecationWarning)

#### This cell creates the power_grid() class which contains the relevant networks, dictionaries, lists, values, and functions necessary to investigate the flow of information through the Western States Power Grid of the United States.  This class can then be called into other Jupyter notebooks.

In [2]:
class power_grid():

    '''Initializing the relevant networks, dictionaries, lists, and values'''
    def __init__(self, network, title, model):
        #Network
        self.network = network
        self.title = title
        self.model = model
        self.pos = nx.spring_layout(self.network)
        self.edge_list = list(self.network.edges())
        self.node_list = list(self.network.nodes())
        self.cluster_coeff = 0.0
        self.global_eff = 0.0
        self.local_eff = 0.0

        #Communities
        self.community_list = nx.community.louvain_communities(self.network)
        self.community_dict = dict(sorted({node: num for num, community in enumerate(self.community_list)
                                            for node in community}.items()))
        self.num_communities = len(self.community_list)
        self.community_cmap = plt.cm.viridis(np.linspace(0, 1, self.num_communities))
        self.community_node_cmap = [self.community_cmap[self.community_dict[node]] for node in self.community_dict]
        self.community_bc_list = []
        self.community_max_bc_index = 0
        self.community_density_list = []
        self.community_max_density_index = 0

        #Components
        self.main_component = nx.empty_graph()
        self.isolated_components = []
        self.component_node_cmap = self.community_node_cmap

        #Betweenness Centralities
        self.edge_bc_dict = {}
        self.edge_bc_list = []
        self.edge_bc_avg = 0.0
        self.edge_bc_std = 0.0
        self.node_bc_dict = {}
        self.node_bc_list = []
        self.node_bc_avg = 0.0
        self.node_bc_std = 0.0

        #Cascade
        self.num_fail_communities = 0
        self.threshold = 0.0
        self.num_initial_adopters = 0
        self.node_fail_list = []
        self.node_safe_list = []
        self.cascade_node_cmap = self.community_node_cmap
        self.cascade_cmap_list = []


    '''This function calculates the global and local efficiency as well as the clustering coefficient of the network'''
    def efficiency(self):
        self.cluster_coeff = round(nx.average_clustering(self.network), 4)
        self.global_eff = round(nx.global_efficiency(self.network), 4)
        self.local_eff = round(nx.local_efficiency(self.network), 4)


    '''This function finds the components of the network and separates the isolated components
       from the main network, which is defined as the largest connected component'''
    def find_components(self):
        components = sorted(list(nx.connected_components(self.network)), key = len, reverse = True)
        subgraphs = list(self.network.subgraph(component).copy() for component in components)
        self.main_component = subgraphs[0]
        if len(subgraphs) != 1:
            self.isolated_components = subgraphs[1:]


    '''This function finds a dictionary, list, average, and standard deviation of the
       betweenness centrality for both the edges and the nodes in the network'''
    def betweenness_centrality(self):
        self.edge_bc_dict = nx.edge_betweenness_centrality(self.network)
        self.edge_bc_list = np.array([value for value in self.edge_bc_dict.values()], dtype = object)
        self.edge_bc_avg = round(np.average(self.edge_bc_list), 4)
        self.edge_bc_std = round(np.std(self.edge_bc_list), 4)
        self.node_bc_dict = nx.betweenness_centrality(self.network)
        self.node_bc_list = np.array([value for value in self.node_bc_dict.values()], dtype = object)
        self.node_bc_avg = round(np.average(self.node_bc_list), 4)
        self.node_bc_std = round(np.std(self.node_bc_list), 4)


    '''This function finds the total betweenness centrality of the nodes in each community in the network'''
    def community_bc(self):
        self.betweenness_centrality()
        for community in self.community_list:
            bc = 0
            for node in community:
                bc += self.node_bc_dict[node]
            self.community_bc_list.append(round(bc, 4))
        self.community_max_bc_index = int(self.community_bc_list.index(max(self.community_bc_list)))


    '''This function finds the density of each community in the network'''
    def community_density(self):
        for community in self.community_list:
            subgraph = self.network.subgraph(list(community))
            self.community_density_list.append(round(nx.density(subgraph), 4))
        self.community_max_density_index = int(self.community_density_list.index(max(self.community_density_list)))


    '''This function deletes edges from N communities in the network with the largest total betweenness centralities'''
    def delete_community_edges(self, N):
        self.num_fail_communities = int(N)
        self.community_bc()
        for index in range(self.num_fail_communities):
            fail_community_index = self.community_bc_list.index(sorted(self.community_bc_list, reverse = True)[index])
            fail_community = self.community_list[fail_community_index]
            for node in fail_community:
                for edge in list(self.network.edges(node)):
                    self.network.remove_edge(*edge)
        self.find_components()
        for component in self.isolated_components:
            for node in component:
                self.component_node_cmap[node] = 'red'


    '''This function deletes N edges with the largest betweenness centralities from the network'''
    def delete_largestbc_edges(self, N):
        self.betweenness_centrality()
        sorted_edge_bc_dict = sorted(self.edge_bc_dict.items(), key = lambda n: n[1])
        sorted_edge_bc_list = [edge[0] for edge in sorted_edge_bc_dict]
        edge_removed_list = sorted_edge_bc_list[-N : ]
        self.network.remove_edges_from(edge_removed_list)
        self.find_components()
        for component in self.isolated_components:
            for node in component:
                self.component_node_cmap[node] = 'red'


    '''This function deletes N random edges from the network'''
    def delete_random_edges(self, N):
        self.initial_adopters = int(N)
        edge_rnd_list = np.unique(np.random.randint(len(self.edge_list), size = self.initial_adopters))
        edge_removed_list = list(self.edge_list[edge] for edge in edge_rnd_list)
        self.network.remove_edges_from(edge_removed_list)
        self.find_components()
        for component in self.isolated_components:
            for node in component:
                self.component_node_cmap[node] = 'red'


    '''This function deletes nodes from N communities in the network with the largest total betweenness centralities'''
    def delete_community_nodes(self, N):
        self.num_fail_communities = int(N)
        self.community_bc()
        node_removed_list = []
        for index in range(self.num_fail_communities):
            fail_community_index = self.community_bc_list.index(sorted(self.community_bc_list, reverse = True)[index])
            fail_community = self.community_list[fail_community_index]
            for node in fail_community:
                self.network.remove_node(node)
                node_removed_list.append(node)
        self.find_components()
        for component in self.isolated_components:
            for node in component:
                self.component_node_cmap[node] = 'red'
        for node in sorted(node_removed_list, reverse = True):
            del self.component_node_cmap[node]


    '''This function deletes N nodes with the largest betweenness centralities from the network'''
    def delete_largestbc_nodes(self, N):
        self.betweenness_centrality()
        sorted_node_bc_dict = sorted(self.node_bc_dict.items(), key = lambda n: n[1])
        sorted_node_bc_list = [node[0] for node in sorted_node_bc_dict]
        node_removed_list = sorted_node_bc_list[-N : ]
        self.network.remove_nodes_from(node_removed_list)
        self.find_components()
        for component in self.isolated_components:
            for node in component:
                self.component_node_cmap[node] = 'red'
        for node in sorted(node_removed_list, reverse = True):
            del self.component_node_cmap[node]


    '''This function deletes N random of nodes from the network'''
    def delete_random_nodes(self, N):
        self.initial_adopters = int(N)
        node_rnd_list = np.unique(np.random.randint(len(self.node_list), size = self.initial_adopters))
        self.network.remove_nodes_from(node_rnd_list)
        self.find_components()
        for component in self.isolated_components:
            for node in component:
                self.component_node_cmap[node] = 'red'
        for node in sorted(node_rnd_list, reverse = True):
            del self.component_node_cmap[node]


    '''This functions simulates a cascade of failures through the network for N initial
       failed communities with the largest total betweenness centralities'''
    def cascade_community(self, N, q):
        self.num_fail_communities = int(N)
        self.threshold = q
        self.community_bc()
        cmap = [self.cascade_node_cmap[index] for index in range(len(self.cascade_node_cmap))]
        self.cascade_cmap_list.append(cmap)
        self.draw_cascade_community(0, self.cascade_cmap_list[0])
        for index in range(self.num_fail_communities):
            fail_community_index = self.community_bc_list.index(sorted(self.community_bc_list, reverse = True)[index])
            fail_community = self.community_list[fail_community_index]
            for node in fail_community:
                self.node_fail_list.append(node)
        self.node_safe_list = [node for node in self.node_list if node not in self.node_fail_list]
        for node in self.node_fail_list:
            self.cascade_node_cmap[node] = 'red'
        cmap = [self.cascade_node_cmap[index] for index in range(len(self.cascade_node_cmap))]
        self.cascade_cmap_list.append(cmap)
        self.draw_cascade_community(1, self.cascade_cmap_list[1])
        num_steps = 1
        while len(self.node_fail_list) <= len(self.node_list):
            check = len(self.node_fail_list)
            new_node_fail_list = []
            for node in self.node_safe_list:
                neighbors = list(nx.neighbors(self.network, node))
                num_fail_neighbors = 0
                for neighbor in neighbors:
                    if neighbor in self.node_fail_list:
                        num_fail_neighbors += 1
                if (num_fail_neighbors / len(neighbors)) >= self.threshold:
                    new_node_fail_list.append(node)
                    self.node_safe_list.remove(node)
            for node in new_node_fail_list:
                self.node_fail_list.append(node)
                self.cascade_node_cmap[node] = 'red'
            if len(self.node_fail_list) == check:
                break
            cmap = [self.cascade_node_cmap[index] for index in range(len(self.cascade_node_cmap))]
            self.cascade_cmap_list.append(cmap)
            num_steps += 1
            self.draw_cascade_community(num_steps, self.cascade_cmap_list[num_steps])


    '''This function simulates a cascade of failures through the network for N
       initial failed nodes with the highest betweenness centralities'''
    def cascade_nodebc(self, N, q):
        self.num_initial_adopters = N
        self.threshold = q
        cmap = [self.cascade_node_cmap[index] for index in range(len(self.cascade_node_cmap))]
        self.cascade_cmap_list.append(cmap)
        self.draw_cascade_nodebc(0, self.cascade_cmap_list[0])
        self.betweenness_centrality()
        sorted_node_bc_dict = sorted(self.node_bc_dict.items(), key = lambda n: n[1])
        sorted_node_bc_list = [node[0] for node in sorted_node_bc_dict]
        self.node_fail_list = sorted_node_bc_list[-N : ]
        self.node_safe_list = sorted_node_bc_list[ : -N]
        for node in self.node_fail_list:
            self.cascade_node_cmap[node] = 'red'
        cmap = [self.cascade_node_cmap[index] for index in range(len(self.cascade_node_cmap))]
        self.cascade_cmap_list.append(cmap)
        self.draw_cascade_nodebc(1, self.cascade_cmap_list[1])
        num_steps = 1
        while len(self.node_fail_list) <= len(self.node_list):
            check = len(self.node_fail_list)
            new_node_fail_list = []
            for node in self.node_safe_list:
                neighbors = list(nx.neighbors(self.network, node))
                num_fail_neighbors = 0
                for neighbor in neighbors:
                    if neighbor in self.node_fail_list:
                        num_fail_neighbors += 1
                if (num_fail_neighbors / len(neighbors)) >= self.threshold:
                    new_node_fail_list.append(node)
                    self.node_safe_list.remove(node)
            for node in new_node_fail_list:
                self.node_fail_list.append(node)
                self.cascade_node_cmap[node] = 'red'
            if len(self.node_fail_list) == check:
                break
            cmap = [self.cascade_node_cmap[index] for index in range(len(self.cascade_node_cmap))]
            self.cascade_cmap_list.append(cmap)
            num_steps += 1
            self.draw_cascade_nodebc(num_steps, self.cascade_cmap_list[num_steps])


    '''This function simulates a cascade of failures through the network for N random initial failed nodes'''
    def cascade_random(self, N, q):
        self.num_initial_adopters = int(N)
        self.threshold = q
        cmap = [self.cascade_node_cmap[index] for index in range(len(self.cascade_node_cmap))]
        self.cascade_cmap_list.append(cmap)
        self.draw_cascade_random(0, self.cascade_cmap_list[0])
        self.node_fail_list = list(np.unique(np.random.randint(len(self.node_list), size = int(N))))
        self.node_safe_list = [node for node in self.node_list if node not in self.node_fail_list]
        for node in self.node_fail_list:
            self.cascade_node_cmap[node] = 'red'
        cmap = [self.cascade_node_cmap[index] for index in range(len(self.cascade_node_cmap))]
        self.cascade_cmap_list.append(cmap)
        self.draw_cascade_random(1, self.cascade_cmap_list[1])
        num_steps = 1
        while len(self.node_fail_list) <= len(self.node_list):
            check = len(self.node_fail_list)
            new_node_fail_list = []
            for node in self.node_safe_list:
                neighbors = list(nx.neighbors(self.network, node))
                num_fail_neighbors = 0
                for neighbor in neighbors:
                    if neighbor in self.node_fail_list:
                        num_fail_neighbors += 1
                if (num_fail_neighbors / len(neighbors)) >= self.threshold:
                    new_node_fail_list.append(node)
                    self.node_safe_list.remove(node)
            for node in new_node_fail_list:
                self.node_fail_list.append(node)
                self.cascade_node_cmap[node] = 'red'
            if len(self.node_fail_list) == check:
                break
            cmap = [self.cascade_node_cmap[index] for index in range(len(self.cascade_node_cmap))]
            self.cascade_cmap_list.append(cmap)
            num_steps += 1
            self.draw_cascade_random(num_steps, self.cascade_cmap_list[num_steps])


    '''This function draws the network'''
    def draw_network(self, position, node_cmap):
        self.efficiency()
        fig, ax = plt.subplots(figsize = (10, 6), dpi = 144)
        fig.tight_layout()
        nx.draw(self.network, position, node_size = 4.5, node_color = node_cmap, with_labels = False)
        ax.plot([], [], linestyle = 'None', label = 'Clustering Coeff. = {}'.format(self.cluster_coeff))
        ax.plot([], [], linestyle = 'None', label = 'Global Eff. = {}'.format(self.global_eff))
        ax.plot([], [], linestyle = 'None', label = 'Local Eff. = {}'.format(self.local_eff))
        ax.set_title('Western States Power Grid of the United States {}'.format(self.title))
        ax.legend(loc = 'upper left', prop = {'size': 10}, handlelength = 0.0, handletextpad = 0.0, frameon = False)
        fig.savefig('c:/Users/Tucker Knaak/Downloads/{}_network.png'.format(self.model), bbox_inches = 'tight')


    '''This functions draws the network for a cascade of failures through the network for
       N initial failed communities with the largest total betweenness centralities'''
    def draw_cascade_community(self, num_steps, node_cmap):
        fig, ax = plt.subplots(figsize = (10, 6), dpi = 144)
        fig.tight_layout()
        nx.draw(self.network, self.pos, node_size = 4.5, node_color = node_cmap, with_labels = False)
        ax.plot([], [], linestyle = 'None', label = 'Step = {}'.format(num_steps))
        if num_steps == 0:
            ax.plot([], [], linestyle = 'None', label = 'Working Nodes = {}'.format(len(self.node_list)))
        else:
            ax.plot([], [], linestyle = 'None', label = 'Working Nodes = {}'.format(len(self.node_safe_list)))
        ax.plot([], [], linestyle = 'None', label = 'Failed Nodes = {}'.format(len(self.node_fail_list)))
        ax.plot([], [], linestyle = 'None', label = 'Threshold = {}'.format(self.threshold))
        if self.num_fail_communities == 1:
            ax.set_title('WSPG - Cascading Failures From 1 Initial Failed Community')
        else:
            ax.set_title('WSPG - Cascading Failures From {} Initial Failed Communities'.format(self.num_fail_communities))
        ax.legend(loc = 'upper left', prop = {'size': 10}, handlelength = 0.0, handletextpad = 0.0, frameon = False)
        fig.savefig('c:/Users/Tucker Knaak/Downloads/cascadecom_N{num_com}_q{threshold}_step{step}.png'.format(
                        num_com = self.num_fail_communities, 
                        threshold = int(self.threshold * 100), 
                        step = num_steps), 
                        bbox_inches = 'tight')
        plt.close()


    '''This function draws the network for the cascade of failures from N random initial failed nodes'''
    def draw_cascade_nodebc(self, num_steps, node_cmap):
        fig, ax = plt.subplots(figsize = (10, 6), dpi = 144)
        fig.tight_layout()
        nx.draw(self.network, self.pos, node_size = 4.5, node_color = node_cmap, with_labels = False)
        ax.plot([], [], linestyle = 'None', label = 'Step = {}'.format(num_steps))
        if num_steps == 0:
            ax.plot([], [], linestyle = 'None', label = 'Working Nodes = {}'.format(len(self.node_list)))
        else:
            ax.plot([], [], linestyle = 'None', label = 'Working Nodes = {}'.format(len(self.node_safe_list)))
        ax.plot([], [], linestyle = 'None', label = 'Failed Nodes = {}'.format(len(self.node_fail_list)))
        ax.plot([], [], linestyle = 'None', label = 'Threshold = {}'.format(self.threshold))
        ax.set_title('WSPG - Cascading Failures From {} Initial Failed Nodes with the Largest BC'.format(
                        self.num_initial_adopters))
        ax.legend(loc = 'upper left', prop = {'size': 10}, handlelength = 0.0, handletextpad = 0.0, frameon = False)
        fig.savefig('c:/Users/Tucker Knaak/Downloads/cascadenodebc_N{num_nodes}_q{threshold}_step{step}.png'.format(
                        num_nodes = self.num_initial_adopters, 
                        threshold = int(self.threshold * 100), 
                        step = num_steps), 
                        bbox_inches = 'tight')
        plt.close()


    '''This function draws the network for the cascade of failures from N random initial failed nodes'''
    def draw_cascade_random(self, num_steps, node_cmap):
        fig, ax = plt.subplots(figsize = (10, 6), dpi = 144)
        fig.tight_layout()
        nx.draw(self.network, self.pos, node_size = 4.5, node_color = node_cmap, with_labels = False)
        ax.plot([], [], linestyle = 'None', label = 'Step = {}'.format(num_steps))
        if num_steps == 0:
            ax.plot([], [], linestyle = 'None', label = 'Working Nodes = {}'.format(len(self.node_list)))
        else:
            ax.plot([], [], linestyle = 'None', label = 'Working Nodes = {}'.format(len(self.node_safe_list)))
        ax.plot([], [], linestyle = 'None', label = 'Failed Nodes = {}'.format(len(self.node_fail_list)))
        ax.plot([], [], linestyle = 'None', label = 'Threshold = {}'.format(self.threshold))
        ax.set_title('WSPG - Cascading Failures From {} Random Initial Failed Nodes'.format(self.num_initial_adopters))
        ax.legend(loc = 'upper left', prop = {'size': 10}, handlelength = 0.0, handletextpad = 0.0, frameon = False)
        fig.savefig('c:/Users/Tucker Knaak/Downloads/cascadernd_N{num_nodes}_q{threshold}_step{step}.png'.format(
                        num_nodes = self.num_initial_adopters, 
                        threshold = int(self.threshold * 100), 
                        step = num_steps), 
                        bbox_inches = 'tight')
        plt.close()


    '''This function creates a table for the total betweenness centrality of the nodes in 
       each community as well as the density of each community in the network'''
    def community_table(self):
        self.community_bc()
        self.community_density()
        table = []
        columns = ['Community', 'Nodes', 'Density', 'Total Node BC']
        cell_colors = []
        for community_index in range(self.num_communities):
            rows = [community_index, len(self.community_list[community_index]),
                    self.community_density_list[community_index], self.community_bc_list[community_index]]
            if community_index == self.community_max_density_index:
                if community_index != self.community_max_bc_index:
                    colors = [self.community_cmap[community_index], 'white', 'red', 'white']
                elif community_index == self.community_max_bc_index:
                    colors = [self.community_cmap[community_index], 'white', 'red', 'red']
            elif community_index == self.community_max_bc_index:
                if community_index != self.community_max_density_index:
                    colors = [self.community_cmap[community_index], 'white', 'white', 'red']
                elif community_index == self.community_max_density_index:
                    colors = [self.community_cmap[community_index], 'white', 'red', 'red']
            else:
                colors = [self.community_cmap[community_index], 'white', 'white', 'white']
            table.append(rows)
            cell_colors.append(colors)
        fig, ax = plt.subplots()
        fig.tight_layout()
        ax.axis('off')
        the_table = ax.table(cellText = table, colLabels = columns, cellColours = cell_colors,
                                loc = 'center', cellLoc = 'center')
        the_table.scale(1.0, 2.0)
        fig.savefig('c:/Users/Tucker Knaak/Downloads/{}_communitytable.png'.format(self.model), bbox_inches = 'tight')
        self.community_bc_list = []
        self.community_max_bc_index = 0
        self.community_density_list = []
        self.community_max_density_index = 0


    '''This function plots the betweenness centralities of both the edges and nodes in the network'''
    def plot_bc(self):
        self.betweenness_centrality()
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize = (10, 12), gridspec_kw = {'height_ratios': [1, 1]})
        fig.tight_layout(pad = 5.0)
        
        #Edge betweenness centrality
        ax1.plot([e for e in range(len(self.edge_bc_list))], self.edge_bc_list, alpha = 0.5, linestyle = 'solid', 
                    linewidth = 0.25, color = 'black', marker = 'None', zorder = 1)
        ax1.scatter([e for e in range(len(self.edge_bc_list))], self.edge_bc_list, s = 4.0, c = 'red', zorder = 2)
        ax1.plot([], [], linestyle = 'None', label = 'AVG = {}'.format(self.edge_bc_avg))
        ax1.plot([], [], linestyle = 'None', label = 'STD = {}'.format(self.edge_bc_std))
        ax1.set_xlabel('Edge')
        ax1.set_ylabel('Edge Betweenness Centrality')
        ax1.set_title('WSPG {}'.format(self.title))
        ax1.legend(loc = 'upper left', prop = {'size': 10}, handlelength = 0.0, handletextpad = 0.0, shadow = True)
        fig.savefig('c:/Users/Tucker Knaak/Downloads/{}_edgebc.png'.format(self.model),
                    bbox_inches = mtransforms.Bbox([[0, 0.5], [1, 1]]).transformed(fig.transFigure - fig.dpi_scale_trans))

        #Node betweenness centrality
        ax2.plot([n for n in range(len(self.node_bc_list))], self.node_bc_list, alpha = 0.5, linestyle = 'solid', 
                    linewidth = 0.25, color = 'black', marker = 'None', zorder = 1)
        ax2.scatter([n for n in range(len(self.node_bc_list))], self.node_bc_list, s = 4.0,
                        c = self.component_node_cmap, zorder = 2)
        ax2.plot([], [], linestyle = 'None', label = 'AVG = {}'.format(self.node_bc_avg))
        ax2.plot([], [], linestyle = 'None', label = 'STD = {}'.format(self.node_bc_std))
        ax2.set_xlabel('Node')
        ax2.set_ylabel('Node Betweenness Centrality')
        ax2.set_title('WSPG {}'.format(self.title))
        ax2.legend(loc = 'upper left', prop = {'size': 10}, handlelength = 0.0, handletextpad = 0.0, shadow = True)
        fig.savefig('c:/Users/Tucker Knaak/Downloads/{}_nodebc.png'.format(self.model),
                    bbox_inches = mtransforms.Bbox([[0, 0], [1, 0.5]]).transformed(fig.transFigure - fig.dpi_scale_trans))
        
        
    '''This functions updates the node color for failed and safe nodes in the cascade of failures through the network 
       for N initial failed communities with the largest total betweenness centralities and draws the network'''
    def update_cascade_community(self, i):
        plt.clf()
        colors = self.cascade_cmap_list[i]
        nodes = nx.draw_networkx_nodes(self.network, self.pos, node_color = colors, node_size = 4.5)
        edges = nx.draw_networkx_edges(self.network, self.pos)
        plt.plot([], [], label = 'Step {}'.format(i))
        if self.num_fail_communities == 1:
            plt.title('WSPG - Cascading Failures From 1 Initial Failed Community')
        else:
            plt.title('WSPG - Cascading Failures From {} Initial Failed Communities'.format(self.num_fail_communities))
        plt.legend(loc = 'upper left', prop = {'size': 12}, handlelength = 0.0, handletextpad = 0.0, frameon = False)
        plt.box(False)
        return nodes,


    '''This function updates the node color for failed and safe nodes in the cascade of failures through the network
       for N initial failed nodes with the largest betweenness centralities and draws the network'''
    def update_cascade_nodebc(self, i):
        plt.clf()
        colors = self.cascade_cmap_list[i]
        nodes = nx.draw_networkx_nodes(self.network, self.pos, node_color = colors, node_size = 4.5)
        edges = nx.draw_networkx_edges(self.network, self.pos)
        plt.plot([], [], label = 'Step {}'.format(i))
        plt.title('WSPG - Cascading Failures From {} Initial Failed Nodes with the Largest BC'.format(
                    self.num_initial_adopters))
        plt.legend(loc = 'upper left', prop = {'size': 12}, handlelength = 0.0, handletextpad = 0.0, frameon = False)
        plt.box(False)
        return nodes,


    '''This function updates the node color for failed and safe nodes in the cascade of failures 
       through the network for N random initial failed nodes and draws the network'''
    def update_cascade_random(self, i):
        plt.clf()
        colors = self.cascade_cmap_list[i]
        nodes = nx.draw_networkx_nodes(self.network, self.pos, node_color = colors, node_size = 4.5)
        edges = nx.draw_networkx_edges(self.network, self.pos)
        plt.plot([], [], label = 'Step {}'.format(i))
        plt.title('WSPG - Cascading Failures From {} Random Initial Failed Nodes'.format(self.num_initial_adopters))
        plt.legend(loc = 'upper left', prop = {'size': 12}, handlelength = 0.0, handletextpad = 0.0, frameon = False)
        plt.box(False)
        return nodes,


    '''This function animates the cascade of failures through the network for N initial
       failed communities with the largest total betweenness centralities'''
    def animate_cascade_community(self):
        fig, ax = plt.subplots(figsize = (10, 6), dpi = 144)
        fig.tight_layout(pad = 2.0)
        ani = animation.FuncAnimation(fig, self.update_cascade_community, frames = len(self.cascade_cmap_list),
                                        interval = 250, blit = True, repeat = False)
        HTML(ani.to_jshtml())
        f = r'c:/Users/Tucker Knaak/Downloads/cascadecom_N{num_com}_q{threshold}.gif'.format(
                num_com = self.num_fail_communities, threshold = int(self.threshold * 100))
        writergif = animation.PillowWriter(fps = 5)
        ani.save(f, writer = writergif)


    '''This function animates the cascade of failures through the network for N
       initial failed nodes with the largest betweenness centralities'''
    def animate_cascade_nodebc(self):
        fig, ax = plt.subplots(figsize = (10, 6), dpi = 144)
        fig.tight_layout(pad = 2.0)
        ani = animation.FuncAnimation(fig, self.update_cascade_nodebc, frames = len(self.cascade_cmap_list),
                                        interval = 250, blit = True, repeat = False)
        HTML(ani.to_jshtml())
        f = r'c:/Users/Tucker Knaak/Downloads/cascadenodebc_N{num_nodes}_q{threshold}.gif'.format(
                num_nodes = self.num_initial_adopters, threshold = int(self.threshold * 100))
        writergif = animation.PillowWriter(fps = 5)
        ani.save(f, writer = writergif)


    '''This function animates the cascade of failures through the network for N random initial failed nodes'''
    def animate_cascade_random(self):
        fig, ax = plt.subplots(figsize = (10, 6), dpi = 144)
        fig.tight_layout(pad = 2.0)
        ani = animation.FuncAnimation(fig, self.update_cascade_random, frames = len(self.cascade_cmap_list),
                                        interval = 250, blit = True, repeat = False)
        HTML(ani.to_jshtml())
        f = r'c:/Users/Tucker Knaak/Downloads/cascadernd_N{num_nodes}_q{threshold}.gif'.format(
                num_nodes = self.num_initial_adopters, threshold = int(self.threshold * 100))
        writergif = animation.PillowWriter(fps = 5)
        ani.save(f, writer = writergif)